# 1-卷积神经网络底层搭建
## 实现一个拥有卷积层和池化层的网络，包含前向传播和反向传播

In [1]:
import numpy as np
import h5py 
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0,4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [3]:
def zero_pad(X,pad):
    """边界填充"""
    X_paded = np.pad(X,(
                        (0,0), # 样本数 不填充
                        (pad,pad), # 图像高度 填充p
                        (pad,pad), # 图像宽度 填充p
                        (0,0)), # 通道数 不填充
                        'constant',constant_values = 0)
    
    return X_paded

In [4]:
# 测试边界填充
x = np.random.randn(4,3,3,2)
x_paded = zero_pad(x,2)
x.shape,x_paded.shape,x[1,1],x_paded[1,1]

((4, 3, 3, 2),
 (4, 7, 7, 2),
 array([[ 0.90085595, -0.68372786],
        [-0.12289023, -0.93576943],
        [-0.26788808,  0.53035547]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]))

In [5]:
def conv_single_step(a_slice_prev,W,b):
    """单步卷积"""
    s = np.multiply(a_slice_prev,W) + b
    Z = np.sum(s)
    
    return Z 

In [7]:
# 测试单步卷积
np.random.seed(1)
a_slice_prev = np.random.randn(4,4,3)
W = np.random.randn(4,4,3)
b = np.random.randn(1,1,1)

Z = conv_single_step(a_slice_prev,W,b)
print(Z)

-23.16021220252078


In [8]:
def conv_forward(A_prev,W,b,hparams):
    """卷积前向传播"""
    (m,n_H_prev,n_W_prev,n_C_prev) = A_prev.shape
    
    (f,f,n_C_prev,n_C) = W.shape
    
    stride = hparams["stride"]
    pad = hparams["pad"]
    
    n_H = int((n_H_prev - f + 2 * pad) / stride) + 1
    n_W = int((n_W_prev - f + 2 * pad) / stride) + 1
    
    Z = np.zeros((m,n_H,n_W,n_C))
    
    A_prev_pad = zero_pad(A_prev,pad)
    
    for i in range(m):
        a_prev_pad = A_prev_pad[i]
        
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:] # 切片取出卷积单元
                    
                    Z[i,h,w,c] = conv_single_step(a_slice_prev,W[:,:,:,c],b[0,0,0,c])
                    
    cache = (A_prev,W,b,hparams)
    
    return Z, cache

In [10]:
# 测试卷积前向传播
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)
hparams = {"pad":2,"stride":1}
Z,cache_conv = conv_forward(A_prev,W,b,hparams)

np.mean(Z),cache_conv[0][1][2][3]

(0.15585932488906465, array([-0.20075807,  0.18656139,  0.41005165]))

In [11]:
def pool_forward(A_prev,hparams,mode="max"):
    """池化层前向传播"""
    (m,n_H_prev,n_W_prev,n_C_prev) = A_prev.shape
    
    f = hparams["f"]
    stride = hparams["stride"]
    
    n_H = int((n_H_prev - f) / stride) + 1
    n_W = int((n_W_prev - f) / stride) + 1
    n_C = n_C_prev
    
    A = np.zeros((m,n_H,n_W,n_C))
    
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    a_slice_prev = A_prev[i,vert_start:vert_end,horiz_start:horiz_end,c] # 切片取出卷积单元
                    
                    if mode == "max":
                        A[i,h,w,c] = np.max(a_slice_prev)
                    elif mode == "average":
                        A[i,h,w,c] = np.mean(a_slice_prev)
                        
                        
    cache = (A_prev,hparams)
    
    return A,cache

In [12]:
# 测试池化前向传播
np.random.seed(1)
A_prev = np.random.randn(2,4,4,3)
hparams = {"f":4,"stride":1}
A,cache = pool_forward(A_prev,hparams,mode="max")
print(A)
A,cache = pool_forward(A_prev,hparams,mode="average")
print(A)

[[[[1.74481176 1.6924546  2.10025514]]]


 [[[1.19891788 1.51981682 2.18557541]]]]
[[[[-0.09498456  0.11180064 -0.14263511]]]


 [[[-0.09525108  0.28325018  0.33035185]]]]
